# Efficient Net - Repurposing/Finetuning
## Introduction

This notebook is an attempt to repurpose and finetune an EfficientNet model to the task of American Sign Language detection for the DSPRO2 project at HSLU.

## Setup
In this section all the necessary libraries are imported.

In [6]:
%pip install -r requirements.txt

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.11.16-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached executing-2.2.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached ipython-9.1.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached kagglehub-0.3.11-py3-none-any.whl.metadata (32 kB)
  Using cached lightning-2.5.1-py3-none-any.whl.metadata (39 kB)
  Using cached lightning_utilities-0.14.3-py3-none-any.whl.metadata (

In [11]:
import wandb
import torch
import torch.nn as nn
import torchvision.models as visionmodels
import torchvision.transforms.v2 as transforms
import lightning as L

from lightning.pytorch.loggers import WandbLogger

import nbformat

from typing import Callable

import os

# Our own modules
import models.sweep_helper as sweep_helper

from datapipeline.asl_image_data_module import ASLImageDataModule
from models.asl_model import ASLModel
from models.training import sweep, train_model

In [12]:
os.environ["WANDB_NOTEBOOK_NAME"] = "./dspro2/efficientnet.ipynb"

## Preprocessing
No general data preprocessing is necessary, however there will be random transforms applied to the images during training. The images are resized to 224x224 pixels, which is the input size of the EfficientNet model. The images are also normalized using the mean and standard deviation of the ImageNet dataset, which is the dataset on which the EfficientNet model was pretrained.

The following cells will show the loading of the dataset and the preparation of the mentioned transforms.

In [13]:
PATH = "/exchange/dspro2/silent-speech/ASL_Pictures_Dataset"

In [14]:
img_size = 224

# See https://pytorch.org/vision/master/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py
# for more examples of transforms

# Open Idea: Grayscale for anti bias


data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.3, saturation=0.3, hue=0.3), # Idea: ColorJitter for anti bias
    transforms.RandomRotation(degrees=5),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet stats
])

/opt/conda/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [15]:
datamodule = ASLImageDataModule(path=PATH, transforms=data_transforms, val_split_folder="Validation", batch_size=32, num_workers=128)

## Models

In [26]:
NUM_CLASSES = 28

In [27]:
class ASLEfficientNetRepurpose(nn.Module):
    def __init__(self, efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float = 0.2, num_classes: int = NUM_CLASSES):
        super().__init__()
        self.model = efficientnet_model
        self.model.requires_grad_(False)
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.model.classifier[1].in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [28]:
class ASLEfficientNetFinetune(ASLEfficientNetRepurpose):
    def __init__(self, efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float = 0.2, unfreeze_features: int = 1, num_classes: int = NUM_CLASSES):
        super().__init__(efficientnet_model, dropout, num_classes)

        assert unfreeze_features > 0, "unfreeze_features must be greater than 0"
        unfreeze_features = min(unfreeze_features, len(self.model.features))

        self.model.features[-unfreeze_features:].requires_grad_(True)

## Training

In [29]:
TUNE_TYPE = "tune_type"
EFFICIENTNET_MODEL = "efficientnet_model"
DROPOUT = "dropout"

In [30]:
def get_pretrained_efficientnet_model(model_type: str):
    if model_type == "b0":
        efficientnet_model = visionmodels.efficientnet_b0(weights=visionmodels.EfficientNet_B0_Weights.DEFAULT)
    elif model_type == "b1":
        efficientnet_model = visionmodels.efficientnet_b1(weights=visionmodels.EfficientNet_B1_Weights.DEFAULT)
    elif model_type == "b2":
        efficientnet_model = visionmodels.efficientnet_b2(weights=visionmodels.EfficientNet_B2_Weights.DEFAULT)
    elif model_type == "b3":
        efficientnet_model = visionmodels.efficientnet_b3(weights=visionmodels.EfficientNet_B3_Weights.DEFAULT)
    elif model_type == "b4":
        efficientnet_model = visionmodels.efficientnet_b4(weights=visionmodels.EfficientNet_B4_Weights.DEFAULT)
    elif model_type == "b5":
        efficientnet_model = visionmodels.efficientnet_b5(weights=visionmodels.EfficientNet_B5_Weights.DEFAULT)
    elif model_type == "b6":
        efficientnet_model = visionmodels.efficientnet_b6(weights=visionmodels.EfficientNet_B6_Weights.DEFAULT)
    elif model_type == "b7":
        efficientnet_model = visionmodels.efficientnet_b7(weights=visionmodels.EfficientNet_B7_Weights.DEFAULT)

    return efficientnet_model

In [31]:
UNFREEZE_FEATURES = "unfreeze_features"

def get_asl_efficientnet_model(type: str, efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float, unfreeze_features: int = 1) -> nn.Module:
    if type == "repurpose":
        model = ASLEfficientNetRepurpose(efficientnet_model, dropout=dropout)
    elif type == "finetune":
        model = ASLEfficientNetFinetune(efficientnet_model, dropout=dropout, unfreeze_features=unfreeze_features)
    else:
        raise ValueError(f"Invalid model type: {type}")

    return model

In [32]:
def get_efficientnet_model_from_config(config: dict) -> nn.Module:
    efficientnet_model = get_pretrained_efficientnet_model(config[EFFICIENTNET_MODEL])
    model = get_asl_efficientnet_model(config[TUNE_TYPE], efficientnet_model, config[DROPOUT], config[UNFREEZE_FEATURES])
    return model

In [33]:
run_id = 0
SEED = 42


def train_efficient_net():
    train_model("efficientnet", get_efficientnet_model_from_config, datamodule, seed=SEED)

In [ ]:
sweep_config = {
    "name": "EfficientNet-B0-B2-Repurpose",
    "method": "bayes",
    "metric": {
        "name": f"{ASLModel.VALID_ACCURACY}",
        "goal": "maximize"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    },
    "parameters": {
        TUNE_TYPE: {
            "values": ["repurpose"]
        },
        EFFICIENTNET_MODEL: {
            "values": ["b0", "b1", "b2"]
        },
        UNFREEZE_FEATURES: {
            "min": 0,
            "max": 8
        },
        DROPOUT: {
            "min": 0.1,
            "max": 0.5
        },
        sweep_helper.OPTIMIZER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "values": [sweep_helper.OptimizerType.ADAM, sweep_helper.OptimizerType.ADAMW, sweep_helper.OptimizerType.RMSPROP]
                },
                sweep_helper.LEARNING_RATE: {
                    "min": 1e-5,
                    "max": 1e-2,
                    "distribution": "log_uniform_values"
                },
                sweep_helper.WEIGHT_DECAY: {
                    "min": 0,
                    "max": 1e-3,
                },
                sweep_helper.MOMENTUM: {
                    "min": 0.8,
                    "max": 0.99
                }
            }
        },
        sweep_helper.LEARNING_RATE_SCHEDULER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "values": [sweep_helper.LearningRateSchedulerType.NONE, sweep_helper.LearningRateSchedulerType.STEP, sweep_helper.LearningRateSchedulerType.EXPONENTIAL, sweep_helper.LearningRateSchedulerType.CONSTANT]
                },
                sweep_helper.STEP_SIZE: {
                    "min": 1,
                    "max": 10
                },
                sweep_helper.GAMMA: {
                    "min": 0.1,
                    "max": 0.9
                },
                sweep_helper.FACTOR: {
                    "min": 0.1,
                    "max": 0.5,
                }
            }
        }
    }
}

In [ ]:
sweep(sweep_config=sweep_config, count=20, training_procedure=train_efficient_net)

In [22]:
finetune_sweep_config_b0 = {
    "name": "EfficientNet-B0-Finetune",
    "method": "bayes",
    "metric": {
        "name": f"{ASLModel.VALID_ACCURACY}",
        "goal": "maximize"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    },
    "parameters": {
        TUNE_TYPE: {
            "value": "finetune"
        },
        EFFICIENTNET_MODEL: {
            "value": "b0"
        },
        UNFREEZE_FEATURES: {
            "min": 1,
            "max": 9
        },
        DROPOUT: {
            "min": 0.1,
            "max": 0.5
        },
        sweep_helper.OPTIMIZER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "value": sweep_helper.OptimizerType.RMSPROP
                },
                sweep_helper.LEARNING_RATE: {
                    "min": 1e-5,
                    "max": 1e-2,
                    "distribution": "log_uniform_values"
                },
                sweep_helper.WEIGHT_DECAY: {
                    "min": 0,
                    "max": 1e-3,
                },
                sweep_helper.MOMENTUM: {
                    "min": 0.8,
                    "max": 0.99
                }
            }
        },
        sweep_helper.LEARNING_RATE_SCHEDULER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "values": [sweep_helper.LearningRateSchedulerType.NONE, sweep_helper.LearningRateSchedulerType.STEP, sweep_helper.LearningRateSchedulerType.EXPONENTIAL]
                },
                sweep_helper.STEP_SIZE: {
                    "min": 1,
                    "max": 10
                },
                sweep_helper.GAMMA: {
                    "min": 0.1,
                    "max": 0.9
                }
            }
        }
    }
}

In [ ]:
sweep(sweep_config=finetune_sweep_config_b0, count=30, training_procedure=train_efficient_net)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.


Create sweep with ID: da3s4woi
Sweep URL: https://wandb.ai/dspro2-silent-speech/silent-speech/sweeps/da3s4woi


wandb: Agent Starting Run: 5ya04w7r with config:
wandb: 	dropout: 0.4882254824597369
wandb: 	efficientnet_model: b0
wandb: 	learning_rate_scheduler: {'gamma': 0.12207377267316807, 'step_size': 8, 'type': 'exponential'}
wandb: 	optimizer: {'learning_rate': 0.00014626705462299463, 'momentum': 0.9600000119859484, 'type': 'rmsprop', 'weight_decay': 0.0003786203337267541}
wandb: 	tune_type: finetune
wandb: 	unfreeze_features: 2
Seed set to 42
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.
wandb: Currently logged in as: v8-luky (dspro2-silent-speech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                    | Params | Mode 
-------------------------------------------------------------------
0 | model          | ASLEfficientNetFinetune | 4.0 M  | train
1 | criterion 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Metric train_accuracy improved by 0.089 >= min_delta = 0.0. New best score: 0.759


Validation: |          | 0/? [00:00<?, ?it/s]

Metric valid_accuracy improved by 0.188 >= min_delta = 0.0. New best score: 0.818
Metric train_accuracy improved by 0.049 >= min_delta = 0.0. New best score: 0.808


Validation: |          | 0/? [00:00<?, ?it/s]

Metric train_accuracy improved by 0.055 >= min_delta = 0.0. New best score: 0.862


In [ ]:
finetune_sweep_config_b1 = {
    "name": "EfficientNet-B1-Finetune",
    "method": "bayes",
    "metric": {
        "name": f"{ASLModel.VALID_ACCURACY}",
        "goal": "maximize"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    },
    "parameters": {
        TUNE_TYPE: {
            "value": "finetune"
        },
        EFFICIENTNET_MODEL: {
            "value": "b0"
        },
        UNFREEZE_FEATURES: {
            "min": 1,
            "max": 9
        },
        DROPOUT: {
            "min": 0.1,
            "max": 0.5
        },
        sweep_helper.OPTIMIZER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "value": sweep_helper.OptimizerType.RMSPROP
                },
                sweep_helper.LEARNING_RATE: {
                    "min": 1e-5,
                    "max": 1e-2,
                    "distribution": "log_uniform_values"
                },
                sweep_helper.WEIGHT_DECAY: {
                    "min": 0,
                    "max": 1e-3,
                },
                sweep_helper.MOMENTUM: {
                    "min": 0.8,
                    "max": 0.99
                }
            }
        },
        sweep_helper.LEARNING_RATE_SCHEDULER: {
            "parameters": {
                sweep_helper.TYPE: {
                    "values": [sweep_helper.LearningRateSchedulerType.NONE, sweep_helper.LearningRateSchedulerType.STEP, sweep_helper.LearningRateSchedulerType.EXPONENTIAL]
                },
                sweep_helper.STEP_SIZE: {
                    "min": 1,
                    "max": 10
                },
                sweep_helper.GAMMA: {
                    "min": 0.1,
                    "max": 0.9
                }
            }
        }
    }
}

In [ ]:
sweep(sweep_config=finetune_sweep_config_b1, count=30, training_procedure=train_efficient_net)

## Evaluation

In [16]:
import wandb

In [17]:
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: v8-luky (v8-luky-university-of-applied-sciences-lucerne) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [18]:
from models.training import ENTITY_NAME, PROJECT_NAME

In [19]:
run = wandb.init(project=PROJECT_NAME, entity=ENTITY_NAME)
artifact = run.use_artifact('dspro2-silent-speech/silent-speech/model-9ktrg7uc:v2', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [20]:
artifact_dir

'/home/jovyan/dspro2/artifacts/model-9ktrg7uc:v2'

In [21]:
checkpoint = torch.load(artifact_dir + "/model.ckpt")

In [22]:
prod_run = artifact.logged_by()
config = prod_run.config

In [34]:
inner_model = ASLEfficientNetFinetune(get_pretrained_efficientnet_model(config["efficientnet_model"]), dropout=config["dropout"], unfreeze_features=config["unfreeze_features"])
model = ASLModel(inner_model, criterion=nn.CrossEntropyLoss(), optimizer=torch.optim.Adam(params=inner_model.parameters())) 

In [35]:
model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [36]:
trainer = L.Trainer(accelerator="auto")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [39]:
trainer.test(model, datamodule)

You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_accuracy_epoch      0.9999700784683228
     test_loss_epoch       0.0010380304884165525
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 0.0010380304884165525,
  'test_accuracy_epoch': 0.9999700784683228}]

In [37]:
import torchvision.datasets as datasets
import lightning as L
from torch.utils.data import Dataset, DataLoader

In [49]:
class Predict(L.LightningDataModule):
    def test_dataloader(self):
        return DataLoader(datasets.ImageFolder("/home/jovyan/Predict"))

In [51]:
trainer.test(model, Predict())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints, B. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

wandb: 
wandb: 🚀 View run exalted-energy-122 at: https://wandb.ai/dspro2-silent-speech/silent-speech/runs/vgpfh9rt
wandb: Find logs at: wandb/run-20250414_193414-vgpfh9rt/logs
